In [1]:
'''
"1367138" : {
    "restaurant": "88곱창-분당점",
    "url": "https://www.yogiyo.co.kr/mobile/#/1367138/",
    "reviews": [
        {"menus": "menu1,menu2,menu3", "review_text": "맛있음"},
        {"menus": "menu1,menu3,menu5", "review_text": "굿"}
    ]
}
'''

'\n"1367138" : {\n    "restaurant": "88곱창-분당점",\n    "url": "https://www.yogiyo.co.kr/mobile/#/1367138/",\n    "reviews": [\n        {"menus": "menu1,menu2,menu3", "review_text": "맛있음"},\n        {"menus": "menu1,menu3,menu5", "review_text": "굿"}\n    ]\n}\n'

In [4]:
import os

from dotenv import load_dotenv
from pymongo import MongoClient

load_dotenv()
MONGO_INITDB_HOST = os.getenv('MONGO_INITDB_HOST')
MONGO_INITDB_PORT = int(os.getenv('MONGO_INITDB_PORT'))
MONGO_INITDB_ROOT_USERNAME  = os.getenv('MONGO_INITDB_ROOT_USERNAME')
MONGO_INITDB_ROOT_PASSWORD  = os.getenv('MONGO_INITDB_ROOT_PASSWORD')

client = MongoClient(
    host=MONGO_INITDB_HOST, 
    port=MONGO_INITDB_PORT,
    username=MONGO_INITDB_ROOT_USERNAME,
    password=MONGO_INITDB_ROOT_PASSWORD,
    authSource='admin'
)

db = client['restaurant_db']
collection = db['restaurant_info']

restaurants_info = list(collection.find({}))
restaurants_info

[{'_id': '522757',
  'restaurant': '청년피자-과천점',
  'reviews': [{'menus': '반반 오리지널L/1(반반 오리지널피자 선택 1((반반)슈퍼슈프림),반반 오리지널피자 선택 2((반반)할라불고기),크러스트 추가(기본)),코카콜라 제로 1.25L/1,청양마요소스/1,갈릭딥핑소스/1',
    'review_text': '과천와서 피자는 항상 청년피자 먹고있어요 맛있습니다'},
   {'menus': '[신메뉴]바질 콰트로치즈 피자/1(피자 사이즈 선택(L사이즈 (2~3인)),크러스트 추가(통치즈크러스트(더욱 꽉 찬)))',
    'review_text': '맛있게 잘먹었습니다. 감사합니다.'},
   {'menus': '반반 프리미엄L/1(반반 프리미엄피자 선택 1((반반)메가쉬림프 갈릭크런치),반반 프리미엄피자 선택 2((반반)뉴욕비프 갈릭크런치),음료 추가(코카콜라 1.25L 추가))',
    'review_text': '바삭바삭하니 식감도 좋고 새우도 탱글하고 맛있게 잘먹었어요👍'},
   {'menus': '반반 오리지널L/1(반반 오리지널피자 선택 1((반반)리얼페페로니),반반 오리지널피자 선택 2((반반)슈퍼슈프림),크러스트 추가(기본),사이드메뉴 추가(미트소스스파게티 추가),음료 추가(코카콜라 1.25L 추가))',
    'review_text': '맛있어요\n청양마요소스에 찍어먹으니 더 맛있네요'},
   {'menus': '진짜감자피자/1(피자 사이즈 선택(L사이즈 (2~3인)),크러스트 추가(기본))',
    'review_text': '맛있게 잘 먹었어요'},
   {'menus': '반반 프리미엄L/1(반반 프리미엄피자 선택 1((반반)메가쉬림프 갈릭크런치),반반 프리미엄피자 선택 2((반반)뉴욕비프 갈릭크런치),피자토핑 추가(파인애플 추가),소스 추가(피클 추가))',
    'review_text': '맛있게잘먹었습니다! 토핑많아 맛나요'},
   {'menus': '진짜감자피자/1(피

In [5]:
'''
"1367138_1" : {
    "restaurant": "88곱창-분당점",
    "menu": "수박주스x3",
    "url": "https://www.yogiyo.co.kr/mobile/#/1367138/",
    "keywords": ["해장", "숙취해소"],
    "embeddings": [
        ...
    ]
}
'''

'\n"1367138_1" : {\n    "restaurant": "88곱창-분당점",\n    "menu": "수박주스x3",\n    "url": "https://www.yogiyo.co.kr/mobile/#/1367138/",\n    "keywords": ["해장", "숙취해소"],\n    "embeddings": [\n        ...\n    ]\n}\n'

In [ ]:
KEYWORDS_BLACKLIST = ['리뷰', 'zㅣ쀼', 'ZI쀼', 'Zl쀼', '리쀼', '찜', '이벤트', '추가', '소스']
KEYWORDS_CONTEXT = [
    '해장', '숙취',
    '다이어트'
]

In [7]:
def is_valid_menu(menu_name):
    return True if not any(keyword in menu_name for keyword in KEYWORDS_BLACKLIST) else False

is_valid_menu('[리쀼] 수박 추가')

False

In [8]:
def extract_keywords(review_text):
    keywords = []

    for word in review_text.split():
        if any(keyword in word for keyword in KEYWORDS_CONTEXT):
            keywords.append(word)
    return keywords

extract_keywords('완전 맛있었어요. 역시 해장에는 수박주스가 답이네요.')

['해장에는']

In [9]:
KEYWORDS_BLACKLIST = ['리뷰', 'zㅣ쀼', 'ZI쀼', 'Zl쀼', '찜', '이벤트', '추가']
KEYWORDS_CONTEXT = [
    '해장', '숙취',
    '다이어트'
]

menu_name2id = {}
menu_db = {}

unique_menus = set()
for index, restaurant in enumerate(restaurants_info):
    menu_name2id = {}
    for reviews in restaurant['reviews']:
        menus = reviews["menus"].split(',')
        review_text = reviews["review_text"]

        # 리뷰에 컨텍스트 관련 키워드 있는 지 확인
        keywords = extract_keywords(review_text)

        # 리뷰에 컨텍스트 관련 키워드 없으면 해당 리뷰 스킵
        if keywords == []:
            continue

        for menu in menus:
            menu_name = menu.split('/')[0]
            # 리뷰 이벤트 관련 메뉴 필터링
            if is_valid_menu(menu_name):
                if not menu_name in menu_name2id:
                    menu_idx = len(menu_name2id)
                    menu_id = '_'.join([restaurant['_id'], str(menu_idx)])
                    menu_name2id[menu_name] = menu_id

                    # Menu DB instances format
                    menu_db[menu_id] = {
                        "restaurant": restaurant["restaurant"],
                        "menu": menu_name,
                        "url": restaurant["url"],
                        "keywords": menu_name.split(),
                        "embeddings": None
                    }
                else:
                    menu_id = menu_name2id[menu_name]

                menu_db[menu_id]["keywords"].extend(keywords)

In [12]:
len(menu_db)
menu_db

{'522757_0': {'restaurant': '청년피자-과천점',
  'menu': '달콤한나의고구마',
  'url': 'https://www.yogiyo.co.kr/mobile/#/522757/',
  'keywords': ['달콤한나의고구마', '해장'],
  'embeddings': None},
 '522757_1': {'restaurant': '청년피자-과천점',
  'menu': '피자도우 변경(고구마체다크러스트)',
  'url': 'https://www.yogiyo.co.kr/mobile/#/522757/',
  'keywords': ['피자도우', '변경(고구마체다크러스트)', '해장'],
  'embeddings': None},
 '1212106_0': {'restaurant': 'SLB샐러드-과천점',
  'menu': '바질 닭가슴살 랩',
  'url': 'https://www.yogiyo.co.kr/mobile/#/1212106/',
  'keywords': ['바질', '닭가슴살', '랩', '다이어트에'],
  'embeddings': None}}

In [13]:
menu_db['522757_0']

{'restaurant': '청년피자-과천점',
 'menu': '달콤한나의고구마',
 'url': 'https://www.yogiyo.co.kr/mobile/#/522757/',
 'keywords': ['달콤한나의고구마', '해장'],
 'embeddings': None}

In [14]:
import sys
sys.path.insert(0, "..")

from utils import get_embedding

for menu_id in menu_db:
    keywords = menu_db[menu_id]["keywords"] + [menu_db[menu_id]["menu"]]
    embedding = get_embedding(" ".join(keywords), model='text-embedding-3-large')
    menu_db[menu_id]["embeddings"] = embedding

In [18]:
menu_db['522757_0']

{'restaurant': '청년피자-과천점',
 'menu': '달콤한나의고구마',
 'url': 'https://www.yogiyo.co.kr/mobile/#/522757/',
 'keywords': ['달콤한나의고구마', '해장'],
 'embeddings': [-0.008069118484854698,
  -0.011118961498141289,
  -0.015125738456845284,
  0.004330900032073259,
  0.036079514771699905,
  0.024065358564257622,
  -0.007359134964644909,
  -0.04425358772277832,
  0.001731742755509913,
  -0.022768866270780563,
  -0.009958292357623577,
  -0.015372688882052898,
  -0.007192443124949932,
  0.0083901546895504,
  0.038129206746816635,
  0.003978995140641928,
  0.028868552297353745,
  0.00010476117313373834,
  0.07201085984706879,
  -0.013853942044079304,
  0.013261259533464909,
  -0.03254811838269234,
  0.02597922831773758,
  0.042994141578674316,
  0.037388354539871216,
  0.00039280616329051554,
  -0.014014460146427155,
  0.028127700090408325,
  0.02667069062590599,
  0.022472525015473366,
  -0.00017421609663870186,
  0.021163687109947205,
  0.008680322207510471,
  0.018360793590545654,
  0.010075594298541546,


In [19]:
db = client['menu_db']
collection = db['menu_info']

In [20]:
for menu_id in menu_db:
    result = collection.update_one({"_id": menu_id}, {"$set": menu_db[menu_id]}, upsert=True)